In [1]:
import pandas as pd
import re
import unicodedata

# Load data and set datatypes

In [2]:
users_df = pd.read_csv("users.csv")

users_df['user_id'] = users_df['user_id'].astype(int)
users_df['name'] = users_df['name'].astype('string')
users_df['email'] = users_df['email'].astype('string')
users_df['signup_date'] = pd.to_datetime(users_df['signup_date'], errors='coerce')

display(users_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      102 non-null    int64         
 1   name         102 non-null    string        
 2   email        102 non-null    string        
 3   signup_date  102 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), string(2)
memory usage: 3.3 KB


None

In [3]:
events_df = pd.read_csv("events.csv")

events_df['event_id'] = events_df['event_id'].astype(int)
events_df['user_email'] = events_df['user_email'].astype('string')
events_df['event_type'] = events_df['event_type'].astype('string')
events_df['timestamp'] = pd.to_datetime(events_df['timestamp'], errors='coerce')

display(events_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   event_id    500 non-null    int64         
 1   user_email  495 non-null    string        
 2   event_type  500 non-null    string        
 3   timestamp   500 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(1), string(2)
memory usage: 15.8 KB


None

# Data Cleaning
- Email cleaning
    - Remove invisible/special characters
    - Remove periods from local part of email from certain email providers
        - Gmail and Protonmail ignore periods in local part of email address
    - Fix domains of email addresses that are missing periods.
        - We need to be care with this step because we cannot just naively add periods to domains that are missing them. To handle this with as little error as possible, we keep a whitelist of popular email domains and their known top-level domains.
    - Strip whitespace
    - Lowercase
- Name cleaning
    - Discard prefixes/suffixes from names
    - Strip whitespace
    - Lowercase

In [4]:
def fix_domain(domain: str) -> str:
    misformed_domain_map = {
        'aolcom': 'aol.com',
        'livecom': 'live.com',
        'yahoocom': 'yahoo.com',
        'gmailcom': 'gmail.com',
        'hotmailcom': 'hotmail.com',
        'outlookcom': 'outlook.com',
        'icloudcom': 'icloud.com',
        'protonmailcom': 'protonmail.com',
        'msncom': 'msn.com',
        'comcastnet': 'comcast.net'
    }

    if domain in misformed_domain_map:
        return misformed_domain_map[domain]
    else:
        return domain

def normalize_email(email: str) -> str:
    if pd.isna(email):
        return pd.NA

    email = unicodedata.normalize("NFKC", email)
    email = re.sub(r'[\u00A0\u200B]', '', email)
    email = email.strip().lower()

    email_parts = email.split('@')
    email_parts = [part.strip() for part in email_parts]

    if len(email_parts) == 2:
        email_parts[1] = fix_domain(email_parts[1])
        
        if email_parts[1] in ('gmail.com', 'protonmail.com'):
            email_parts[0] = email_parts[0].replace('.', '')
        email = '@'.join(email_parts)
    
    return email


def clean_name(name: str) -> str:
    prefixes = ['mr', 'mrs', 'ms', 'dr', 'miss']
    suffixes = ['jr', 'sr', 'ii', 'iii', 'iv']

    words = name.lower().strip().split()
    cleaned = [w for w in words if w.strip('.') not in prefixes + suffixes]
    return ' '.join(cleaned)

users_df['name'] = users_df['name'].apply(clean_name).astype("string")
users_df['email'] = users_df['email'].apply(normalize_email).astype("string")
events_df['user_email'] = events_df['user_email'].apply(normalize_email).astype("string")


In [5]:
# Sanity checking the data
display(users_df)
display(events_df)

,user_id,name,email,signup_date
0,0,danielle johnson,daniellejohnson@gmail.com,2024-09-27
1,1,jeffery wagner,jeffery.wagner@icloud.com,2024-07-26
2,2,amy robinson,amy.robinson@outlook.com,2023-04-12
3,3,linda wolfe,lindawolfe@gmail.com,2025-03-07
4,4,matthew davis,matthew.davis@hotmail.com,2024-06-22
...,...,...,...,...
97,97,gregory estes white,gregoryestes@gmail.com,2024-02-19
98,98,brent wright white,brent.wright@msn.com,2024-11-13
99,99,dana cannon,dana.cannon@outlook.com,2023-10-25
100,100,rick daniels-brown,rick.daniels@outlook.com,2024-02-09


,event_id,user_email,event_type,timestamp
0,1,danielburton@protonmail.com,logout,2023-09-10 20:32:58.421093
1,2,jenniferevans@msn.com,purchase,2023-12-03 23:55:36.554694
2,3,jessicanunez@gmail.com,logout,2024-10-16 07:20:12.854738
3,4,tanyahouse@protonmail.com,password_reset,2024-01-11 04:28:54.790473
4,5,lisa.evans@msn.com,login,2024-10-09 13:41:53.942650
...,...,...,...,...
495,496,justin.jordan@outlook.com,updated_profile,2024-01-25 01:48:58.194437
496,497,devon.hall@comcast.net,logout,2023-08-25 21:27:20.373314
497,498,tanyahouse@protonmail.com,viewed_product,2023-07-07 11:28:07.010142
498,499,barbarascott@outlook.com,login,2024-12-18 17:08:01.895173


# Split name
Assumptions made after removing prefixes/suffixes:
- First word in `name` is first_name
- Last word in `name` is last_name
- Any word in the middle is considered a middle name
- If we only get 1 name, we will use it for the first_name

In [6]:
def split_name(name: str) -> str:
    words = name.strip().split()
    if len(words) >= 2:
        return pd.Series([words[0], words[-1]])
    elif len(words) == 1:
        return pd.Series([words[0], ''])
    else:
        return pd.Series(['', ''])

users_df[['first_name', 'last_name']] = users_df['name'].apply(split_name).astype("string")

In [7]:
# Sanity Checks on datatypes, and if we have any empty names

display(users_df.info())
display(users_df[users_df['first_name'].isna() | (users_df['first_name'].str == "")])
display(users_df[users_df['last_name'].isna() | (users_df['last_name'].str == "")])

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      102 non-null    int64         
 1   name         102 non-null    string        
 2   email        102 non-null    string        
 3   signup_date  102 non-null    datetime64[ns]
 4   first_name   102 non-null    string        
 5   last_name    102 non-null    string        
dtypes: datetime64[ns](1), int64(1), string(4)
memory usage: 4.9 KB


None

,user_id,name,email,signup_date,first_name,last_name


,user_id,name,email,signup_date,first_name,last_name


# Join the data

In [8]:
joined_events_df = pd.merge(events_df, users_df, left_on='user_email', right_on='email', how='left', indicator=True)

display(joined_events_df)

,event_id,user_email,event_type,timestamp,user_id,name,email,signup_date,first_name,last_name,_merge
0,1,danielburton@protonmail.com,logout,2023-09-10 20:32:58.421093,14.0,daniel burton,danielburton@protonmail.com,2023-05-24,daniel,burton,both
1,2,jenniferevans@msn.com,purchase,2023-12-03 23:55:36.554694,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
2,3,jessicanunez@gmail.com,logout,2024-10-16 07:20:12.854738,29.0,jessica nunez,jessicanunez@gmail.com,2024-01-27,jessica,nunez,both
3,4,tanyahouse@protonmail.com,password_reset,2024-01-11 04:28:54.790473,32.0,tanya house,tanyahouse@protonmail.com,2024-03-28,tanya,house,both
4,5,lisa.evans@msn.com,login,2024-10-09 13:41:53.942650,95.0,lisa evans-lee,lisa.evans@msn.com,2023-07-15,lisa,evans-lee,both
...,...,...,...,...,...,...,...,...,...,...,...
495,496,justin.jordan@outlook.com,updated_profile,2024-01-25 01:48:58.194437,67.0,justin jordan,justin.jordan@outlook.com,2024-03-12,justin,jordan,both
496,497,devon.hall@comcast.net,logout,2023-08-25 21:27:20.373314,79.0,devon hall,devon.hall@comcast.net,2023-05-24,devon,hall,both
497,498,tanyahouse@protonmail.com,viewed_product,2023-07-07 11:28:07.010142,32.0,tanya house,tanyahouse@protonmail.com,2024-03-28,tanya,house,both
498,499,barbarascott@outlook.com,login,2024-12-18 17:08:01.895173,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only


# Flag Anomalies
- Output event_id's that do not match to any user
- Flag duplicate emails in users.csv

In [9]:
unmatched_events_df = joined_events_df[joined_events_df['_merge'] == 'left_only']

display(unmatched_events_df)

,event_id,user_email,event_type,timestamp,user_id,name,email,signup_date,first_name,last_name,_merge
1,2,jenniferevans@msn.com,purchase,2023-12-03 23:55:36.554694,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
84,85,chadbeck@comcast.net,purchase,2023-12-13 20:28:49.048561,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
98,99,<NA>,purchase,2023-06-16 05:11:23.034739,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
101,102,nancybaker@icloud.com,viewed_product,2023-11-18 22:04:48.708847,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
107,108,michelewalker@msn.com,logout,2023-12-06 21:05:50.273419,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
223,224,davidparker@outlook.com,password_reset,2023-08-19 19:36:13.211571,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
280,281,<NA>,purchase,2025-02-10 15:34:22.668397,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
285,286,phyllismanning@msn.com,purchase,2024-11-06 07:49:56.047680,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
290,291,josephbowers@comcast.net,password_reset,2023-11-01 20:25:13.251929,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only
301,302,michellecherry@icloud.com,updated_profile,2023-11-30 07:04:08.783915,NaN,<NA>,<NA>,NaT,<NA>,<NA>,left_only


In [10]:
duplicate_emails = duplicated_emails = users_df[users_df['email'].duplicated()]

display(duplicate_emails)

,user_id,name,email,signup_date,first_name,last_name


# Summary of number of events per user

In [11]:
events_summary = joined_events_df.groupby('user_id')['event_type'].count().reset_index()
events_summary.columns = ['user_id', 'num_events']
events_summary['user_id'] = events_summary['user_id'].astype(int)

events_summary = events_summary.sort_values(by='num_events', ascending=False)

display(events_summary)

,user_id,num_events
91,91,12
92,92,12
29,29,9
53,53,8
95,95,8
...,...,...
60,60,2
94,94,2
70,70,2
43,43,1


# Write output to `joined_events.csv`
Note: Only output records that were found in both users and events (inner join results)

In [12]:
output_df = joined_events_df[joined_events_df['_merge'] == 'both']
output_df = output_df[['user_id', 'first_name', 'last_name', 'event_type', 'timestamp', 'signup_date']]

output_df.to_csv('joined_events.csv', index=False)